## How to work with flight phase segmentation files

In [52]:
import datetime
import eurec4a

In [42]:
meta = eurec4a.get_flight_segments()
meta.keys()

dict_keys(['HALO'])

### 1. map `segment_id`s to `segment`s
This can be useful for further queries based on specific segment properties.
In addition to the original properties in each `segment`, the `platform_id` and `flight_id` are also stored.

In [61]:
segments = [{**s,
             "platform_id": platform_id,
             "flight_id": flight_id
            }
            for platform_id, flights in meta.items()
            for flight_id, flight in flights.items()
            for s in flight["segments"]
           ]

In [62]:
segments_by_segment_id = {s["segment_id"]: s for s in segments}

### 2. list flight segmentation `kinds`
A segment is an object which includes at minimum a `segment_id`, `name`, `start` and `end` time.

In [63]:
kinds = set(k for s in segments for k in s["kinds"])
kinds

{'baccardi_calibration',
 'circle',
 'circle_break',
 'circling',
 'clover_leg',
 'clover_turn',
 'lidar_leg',
 'radar_calibration_tilted',
 'radar_calibration_wiggle',
 'straight_leg',
 'super_curtain'}

### 3. List of common properties in all `segments`

In [64]:
set.intersection(*(set(s.keys()) for s in segments))

{'dropsondes',
 'end',
 'flight_id',
 'irregularities',
 'kinds',
 'name',
 'platform_id',
 'segment_id',
 'start'}

In [65]:
segment_ids_by_kind = {kind: [segment["segment_id"]
                              for segment in segments
                              if kind in segment["kinds"]]
    for kind in kinds
}

### 4. Examples: 
* total number of all circles flown during EUREC4A / ATOMIC

In [43]:
len(segment_ids_by_kind["circle"])

72

* How much time did HALO and P3 spend circling?

In [69]:
circling_time = sum([s["end"] - s["start"]
                     for s in segments
                     if "circling" in s["kinds"]
                    ], datetime.timedelta())
circling_time

datetime.timedelta(days=3, seconds=29188)

* get `segment_id` for all circles on a given day sorted by the start time, here February 5

In [73]:
segments_ordered_by_start_time = list(sorted(segments, key=lambda s: s["start"]))

In [75]:
circles_Feb05 = [s["segment_id"]
                 for s in segments_ordered_by_start_time
                 if "circle" in s["kinds"]
                 and s["start"].date() == datetime.date(2020,2,5)
                 and s["platform_id"] == "HALO"
                ]
circles_Feb05

['HALO-0205_c1',
 'HALO-0205_c2',
 'HALO-0205_c3',
 'HALO-0205_c4',
 'HALO-0205_c5',
 'HALO-0205_c6']

* select all `dropsondes` with the quality flag `GOOD` from the first circle on February 5.

In [76]:
circles_Feb05[0]

'HALO-0205_c1'

In [77]:
segments_by_segment_id[circles_Feb05[0]]["dropsondes"]["GOOD"]

['HALO-0205_s01',
 'HALO-0205_s02',
 'HALO-0205_s03',
 'HALO-0205_s04',
 'HALO-0205_s05',
 'HALO-0205_s06',
 'HALO-0205_s07',
 'HALO-0205_s08',
 'HALO-0205_s09',
 'HALO-0205_s10',
 'HALO-0205_s12']